In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
import json

from src.input_pipeline import Pipeline

# Get images and boxes

In [ ]:
tf.reset_default_graph()

params = json.load(open('config.json'))
input_pipeline_params = params['input_pipeline_params']

pipeline = Pipeline(
    'data/val.tfrecords',
    batch_size=24, image_size=(640, 360),
    repeat=True, shuffle=True, 
    augmentation=input_pipeline_params
)
features, labels = pipeline.get_batch()

In [ ]:
with tf.Session() as sess:
    sess.run(pipeline.init)
    I, B, L, N = sess.run([
        features['images'], labels['boxes'],
        labels['labels'], labels['num_boxes']
    ])

# Show an augmented image with boxes

In [ ]:
def draw_boxes(image, boxes, labels):
    image_copy = image.copy()
    draw = ImageDraw.Draw(image_copy, 'RGBA')
    width, height = image.size

    for box, label in zip(boxes, labels):
        ymin, xmin, ymax, xmax = box
        xmin, xmax = width*xmin, width*xmax
        ymin, ymax = height*ymin, height*ymax

        text = str(label)
        fill = (255, 255, 255, 45)
        outline = 'black'

        draw.rectangle(
            [(xmin, ymin), (xmax, ymax)],
            fill=fill, outline=outline
        )
        draw.rectangle(
            [(xmin, ymin), (xmin + 4*len(text) + 4, ymin + 10)],
            fill='white', outline='white'
        )
        draw.text((xmin + 1, ymin + 1), text, fill='red')

    return image_copy

In [ ]:
# choose an image
i = 6
image = Image.fromarray((np.transpose(I[i], [1, 2, 0])*255.0).astype('uint8'))
num_boxes = N[i]
boxes = B[i][:num_boxes]
classes = L[i][:num_boxes]

draw_boxes(image, boxes, classes)